# Airport Decision Architecture & Transformation Analysis (Flagship)

This notebook supports the repository’s flagship **Business Architecture & Decision Intelligence** narrative.

It is deliberately designed for **airport / airline leadership conversations**:
- *What decisions matter most?*
- *Where should governance and transformation attention sit?*
- *Where does analytics create leverage — and where does it not?*

The aim is not operational optimisation theatre. The aim is **decision prioritisation under structural constraints**.


## Strategic question

**Which airport decision domains create disproportionate value or risk, and how should leadership prioritise data, analytics, and transformation investment?**


## Dataset overview

The dataset models **decision domains** across multiple airport archetypes and captures:

- Financial exposure (annual cost exposure)  
- Customer & airline experience impact  
- Regulatory constraint level  
- Data maturity and automation feasibility  
- Sustainability dependency  
- Cross-stakeholder dependency  

The intent is **comparative prioritisation**, not forecasting.


In [ ]:
import pandas as pd

df = pd.read_csv("../data/airport_decision_architecture_dataset.csv")
df.head()

## Architecture-level aggregation (decision domains)

For business architecture work, we prioritise decisions by **decision domain**, not by airport.


In [ ]:
summary = df.groupby("decision_domain", as_index=False).agg(
    cost_exposure=("annual_cost_exposure_gbp_m", "mean"),
    experience_impact=("customer_experience_impact", "mean"),
    regulatory=("regulatory_constraint_level", "mean"),
    automation=("automation_feasibility", "mean"),
    data_maturity=("data_maturity_level", "mean"),
    sustainability=("sustainability_dependency", "mean"),
    cross_stakeholder=("cross_stakeholder_dependency", "mean"),
    decision_volume=("annual_decision_volume", "mean"),
)
summary.sort_values(["cost_exposure","experience_impact"], ascending=False).head(10)

## Decision Architecture Matrix (2×2)

This is the primary executive artefact.

- **X-axis:** Customer & airline experience impact  
- **Y-axis:** Annual cost exposure  
- **Quadrants:** Provide a decision-classification lens (not a performance score)

**Key design choice:** we avoid bubble charts and colour gradients to maximise readability and boardroom usefulness.


In [ ]:
import matplotlib.pyplot as plt

# Prepare short labels (A, B, C...) to keep the matrix readable.
matrix = summary[["decision_domain","cost_exposure","experience_impact"]].copy()
matrix = matrix.sort_values(["cost_exposure","experience_impact"], ascending=[False, False]).reset_index(drop=True)
matrix["short_label"] = [chr(ord("A") + i) for i in range(len(matrix))]

# Thresholds for the 2x2 (median works well as a neutral split)
x_mid = matrix["experience_impact"].median()
y_mid = matrix["cost_exposure"].median()

fig, ax = plt.subplots(figsize=(11, 7))

ax.scatter(
    matrix["experience_impact"],
    matrix["cost_exposure"],
    s=90,
    alpha=0.95,
    edgecolors="black",
    linewidths=0.6
)

ax.axvline(x=x_mid, linestyle="--", linewidth=1, color="grey")
ax.axhline(y=y_mid, linestyle="--", linewidth=1, color="grey")

for _, r in matrix.iterrows():
    ax.annotate(
        r["short_label"],
        (r["experience_impact"], r["cost_exposure"]),
        textcoords="offset points",
        xytext=(8, 6),
        ha="left",
        va="bottom",
        fontsize=10,
        fontweight="bold"
    )

ax.set_xlabel("Customer & Airline Experience Impact (Low → High)", fontsize=11)
ax.set_ylabel("Annual Cost Exposure (£m) (Low → High)", fontsize=11)
ax.set_title("Airport Decision Architecture Matrix", fontsize=15, fontweight="bold", pad=12)

x_min, x_max = ax.get_xlim()
y_min, y_max = ax.get_ylim()

ax.text(x_min, y_max, "STABILISE", fontsize=10, fontweight="bold", va="top", ha="left")
ax.text(x_max, y_max, "BOARD / SYSTEM DECISIONS", fontsize=10, fontweight="bold", va="top", ha="right")
ax.text(x_min, y_min, "INDUSTRIALISE", fontsize=10, fontweight="bold", va="bottom", ha="left")
ax.text(x_max, y_min, "OPTIMISE & GOVERN", fontsize=10, fontweight="bold", va="bottom", ha="right")

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()

# Save for README usage
plt.savefig("../visuals/insight_snapshot.png", dpi=240)
plt.show()

# Print a legend mapping for the README
legend = matrix[["short_label","decision_domain"]].sort_values("short_label")
legend

## A simple leverage score (supporting lens)

The matrix is the executive artefact. This scoring is supporting evidence to help structure discussion.

We favour a lens that blends:
- cost exposure  
- experience impact  
- cross-stakeholder dependency  
- regulatory constraint  
- sustainability dependency  

This does not produce *the answer*. It produces **priority candidates** for leadership attention and sequencing.


In [ ]:
score = summary.copy()

score["decision_leverage_score"] = (
    score["cost_exposure"].rank(pct=True) +
    score["experience_impact"].rank(pct=True) +
    score["cross_stakeholder"].rank(pct=True) +
    score["regulatory"].rank(pct=True) +
    score["sustainability"].rank(pct=True)
)

score.sort_values("decision_leverage_score", ascending=False)[
    ["decision_domain","decision_leverage_score","cost_exposure","experience_impact","regulatory","sustainability","cross_stakeholder","automation","data_maturity"]
].head(12)

## Interpretation: what leadership should do with this

A decision architecture lens leads to three transformation tracks:

### 1) Stabilise  
Decisions where variability introduces risk without meaningful upside.
- Reduce noise, standardise controls, avoid unnecessary change.

### 2) Optimise & Govern  
Decisions where repeatability + cross-functional dependency justify analytics investment.
- Clarify ownership, instrument data, introduce scenario capability, build governance cadence.

### 3) Board / System Decisions  
Decisions with structural consequences, regulatory dependency, and multi-year trade-offs.
- Board sponsorship, stakeholder alignment, sequencing, and capital planning matter more than dashboards.

The point: **sequence transformation by decision leverage**, not by technology availability.


## Outputs to use in the README

- `../visuals/insight_snapshot.png` is automatically generated by this notebook  
- The label legend table can be pasted into the README under the image to make the matrix instantly interpretable
